In [200]:
import xml.etree.ElementTree as ET
import os
import pathlib

from unidecode import unidecode

In [201]:
def gerar_arquivo(arquivo, xml):
    tree = ET.parse(xml)

    root = tree.getroot()

    for child in root.findall('DADOS-GERAIS'):
        arquivo.write("Nome: "+child.attrib["NOME-COMPLETO"]+"\n\n")

    for child in root.findall("PRODUCAO-BIBLIOGRAFICA"):
        string = ""
        for artigos in child.findall("ARTIGOS-PUBLICADOS"):
            for artigo in artigos.findall("ARTIGO-PUBLICADO"):
                for dados_basicos in artigo.findall("DADOS-BASICOS-DO-ARTIGO"):
                    string += "\t<Artigo >\n"
                    
                    titulo_do_artigo = dados_basicos.attrib["TITULO-DO-ARTIGO"]
                    ano_do_artigo = dados_basicos.attrib["ANO-DO-ARTIGO"]
                    doi = dados_basicos.attrib["DOI"]
                    home_page_do_trabalho = dados_basicos.attrib["HOME-PAGE-DO-TRABALHO"]
                    idioma_do_artigo = dados_basicos.attrib["IDIOMA"]

                    #test_string = ""
                    
                    #for i in dados_basicos.attrib.keys():
                        #test_string += i+": "+dados_basicos.attrib[i]

                    #arquivo.write(test_string)

                    string += "\t\tTitulo: "+titulo_do_artigo+"\n"
                    string += "\t\tAno: "+ano_do_artigo+"\n"
                    

                    if home_page_do_trabalho:
                        home_page = home_page_do_trabalho.replace("[", "").replace("]", "")

                        if "doi:" in home_page:
                            aux = home_page.split("doi:")
                            home_page = aux[0]
                            doi = aux[1]

                        if home_page:
                            string += "\t\tHome page: "+home_page+"\n"
                    
                    if doi:
                        string += "\t\tDOI: "+doi+"\n"
                    


                    string += "\t\tIdioma: "+idioma_do_artigo+"\n"


                string += "\n"


                for detalhamento in artigo.findall("DETALHAMENTO-DO-ARTIGO"):

                    titulo_da_revista = detalhamento.attrib["TITULO-DO-PERIODICO-OU-REVISTA"]
                    issn = detalhamento.attrib["ISSN"]
                    volume = detalhamento.attrib["VOLUME"]
                    serie = detalhamento.attrib["SERIE"]
                    pagina_inicial = detalhamento.attrib["PAGINA-INICIAL"]
                    pagina_final = detalhamento.attrib["PAGINA-FINAL"]

                    if titulo_da_revista:
                        string += "\t\tTitulo da revista: " +titulo_da_revista+"\n"
                    
                    if issn:
                        string += "\t\tISSN: " +issn+"\n"

                    if volume:
                        string += "\t\tVolume: " +volume+"\n"

                    if serie:
                        string += "\t\tSerie: " + serie+"\n"

                    if pagina_inicial:
                        string += "\t\tPagina inicial: " +pagina_inicial+"\n"
                    
                    if pagina_final:
                        string += "\t\tPagina final: " +pagina_final+"\n"


                string += ("\n")


                string += "\t\tAutor(es): "
                for autores in artigo.findall("AUTORES"):
                    string += autores.attrib["NOME-COMPLETO-DO-AUTOR"]+", "
                    
                    
                string += "\n\n"

                arquivo.write(string)

def xml_get_first_name(xml):
    tree = ET.parse(xml)
    root = tree.getroot()
    
    nome = ""

    for child in root.findall('DADOS-GERAIS'):
        nome = child.attrib["NOME-COMPLETO"]
    
    primeiro_nome = nome.split(" ")
    
    return primeiro_nome[0]

In [202]:
diretorio_entrada = "./xmls/"
diretorio_saida = "./saidas/"

if not os.path.exists(diretorio_entrada):
    os.makedirs(diretorio_entrada)

if not os.path.exists(diretorio_saida):
    os.makedirs(diretorio_saida)

arquivo = open(diretorio_saida+"todos_curriculos.txt", 'w')

for filename in os.listdir(diretorio_entrada):
    
    f = os.path.join(diretorio_entrada, filename)

    if os.path.isfile(f):
        
        if(pathlib.Path(f).suffix == '.xml'):

            print("Gerando o curriculo do arquivo: "+f)
            nome_do_arquivo = xml_get_first_name(""+f)

            other_file = open(diretorio_saida+"curriculo_"+unidecode(nome_do_arquivo.lower())+".txt", "w")

            gerar_arquivo(other_file, ""+f)
            gerar_arquivo(arquivo, ""+f)

            other_file.close()

arquivo.close()


Gerando o curriculo do arquivo: ./xmls/curriculo.xml
Gerando o curriculo do arquivo: ./xmls/curriculo_1.xml
Gerando o curriculo do arquivo: ./xmls/curriculo_2.xml
Gerando o curriculo do arquivo: ./xmls/curriculo_3.xml
Gerando o curriculo do arquivo: ./xmls/curriculo_4.xml
Gerando o curriculo do arquivo: ./xmls/curriculo_5.xml
